# <mark> <b> > 2.0 </b> Pipeline de Extracao de dados de documentos - NLP </mark>

<b>2_extract_pipeline_NLP_V0.ipynb</b>    |     Atual notebook com as funçoes para processamento de documentos com soluçao NLP

### Modules e config

In [1]:
import os
import sys
import shutil
import platform
import subprocess
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import spacy
from spacy import displacy
import pandas as pd
from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy.tokens import Token
from spacy.language import Language

nlp = spacy.load("pt_core_news_sm")


import logging

# Modulos da solucao
# import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron
import modules.nova_extracao_pdf_pesquisavel as novaextra 
import modules.trata_model as tmod
import modules.trata_pdf as tpdf
import modules.utils as utl

In [2]:
ner = nlp.remove_pipe('ner')
ruler = nlp.add_pipe("entity_ruler")

In [3]:
# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'



# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"



# 6. IMPORTANTE - MUDOU - Path para gestao de imagens resized
image_resized_path = "processamentos/temp/images/processadas"


#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


####Config Processamento Pipeline

# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"







# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# 7. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 8. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"


#### paths de objetos para criacao/gestao (dicionarios/datasets)
cnae_dict_path = "pipeline_extracao_documentos/6_geral_administacao/datasets/CNAE_X_ITEM_SERVICO_PREFEITURAS.xlsx"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

#Modelo atual
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

# logging.basicConfig(
#     filename='config/log_ocorrencias.log',
#     level=logging.INFO, 
#     format='%(asctime)s - %(levelname)s - %(message)s', 
#     datefmt='%d/%m/%Y %H:%M:%S'
# )

# logging.info("kernel reiniciado")

'pt_BR.utf8'

### Funcoes originais

<h4><b>A.</b> Funcoes de Imagem </h4>

In [4]:
# XXX IMPORTANTE - ESTA E A FUNCAO PARA SER UTILIZADA: POIS CONVERTE PARA CINZA E RESIZE: (4134, 5846)
def convert_resize_gray(original_file_name, file_path, image_resized_path):

    name_image = utl.conv_filename_no_ext(original_file_name)
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
    imagem_gray = resized_pages[0].convert('L')
    imagem_gray.save(image_resized_name, 'JPEG')

    return  imagem_gray, image_resized_name

# XXX Pequenos mas poderosos
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por', config='--psm 6')
    return texto_extraido 


# 5. XXX Ajusta textoYYY
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

<h4><b>B.</b> Funcoes de Frames </h4>

In [5]:
# funçao importante para buscar coordenadas do frame em funçao do contexto
def get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo):
    
    row_frame = utl.filtrar_df(frames_nf_v4_df, model=model_map, context_mapping=context_mapping, type=tipo)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates



# XXXpara buscar melhor as coordendas dos FRAMES
def get_coordinates_filter(pdf_pesquisavel_map, model, tipo, label, section):
    
    row_frame = utl.filtrar_df(frames_nf_v4_df, model=model, type=tipo, label=label, section_json=section)
    
    # Verificando se row_frame não está vazio
    if not row_frame.empty:
        # Acessando a primeira linha do DataFrame filtrado e depois acessando as colunas
        coodinates = [((row_frame.iloc[0]['x0_p'], row_frame.iloc[0]['y0_p'], row_frame.iloc[0]['x1_p'], row_frame.iloc[0]['y1_p']) if pdf_pesquisavel_map else (row_frame.iloc[0]['x0'], row_frame.iloc[0]['y0'], row_frame.iloc[0]['x1'], row_frame.iloc[0]['y1']))]
    else:
        # Retornando uma tupla de valores NaN se o DataFrame filtrado estiver vazio
        coodinates = [(float('nan'), float('nan'), float('nan'), float('nan'))]
    
    return coodinates

<h4><b>C.</b> Funcoes de Processamento e Extracao </h4>

In [6]:
# 0.A Dados iniciais - PDF PESQUISAVEL	
def pesquisa_prefeitura_pdf_pesquisavel(idx, row, row_info, map_directory, original_file_name, file_path, debug):    
    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    if debug:
        print(f'\ndentro da funçao: pesquisa_prefeitura_pdf_pesquisavel: doc.:{original_file_name} | diretorio: {map_directory}  text: \n\n{text}\n\n')
    
    if text:
       page_number = 0
       #print(page_number)
    else:
       page_number = 1
       #print(page_number)
    
    pdf_document.close()
   
    return text




# 0. INFOMACOES INICIAIS - RASTER PDF
def processar_dados_iniciais(idx, row, row_info, section, map_directory, original_file_name, file_path, debug):
    
    # lista_texto_extraido = []

    nf_dados_doc = {}
    nf_dados_doc['secao'] = section
    pdf_pesquisavel = None
    extracted_txt = pesquisa_prefeitura_pdf_pesquisavel(idx, row, row_info, map_directory, original_file_name, file_path, debug)
    if debug:
        print(f'\n1. funcao: processar_dados_iniciais: doc.:{original_file_name} | diretorio: {map_directory} apos funcao: pesquisa_prefeitura_pdf_pesquisavel: extracted_txt:\n{extracted_txt}\n\n')
    
    if extracted_txt:
        pdf_pesquisavel = True
        print(f'extracted_txt: {extracted_txt} - portanto pdf_pesquisavel: {pdf_pesquisavel} ')
        
    else:
        pdf_pesquisavel = False
        print(f'extracted_txt: {extracted_txt} - portanto pdf_pesquisavel: {pdf_pesquisavel} ') 
       
       
        # WTF
        x0 = 220
        y0 = 0
        x1= 3858
        y1 = 1572
        
        # usando novo processo que gera o arquivo "on the fly" imagem_gray (converte PDF para imagem de tamanho grande (4134, 5846) - torna-a cinza e a salva)
        imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)
        extracted_txt = extract_text_PIL(imagem_gray, (x0, y0, x1, y1))
        #print(f'extracted_txt: {extracted_txt}')
        if debug:
            print(f'\n2. funcao: processar_dados_iniciaisdoc.:{original_file_name} | diretorio: {map_directory}  apos : extract_text_PIL: extracted_txt:\n{extracted_txt}\n\n')
    
    nf_dados_doc['file_name'] = original_file_name    
    nf_dados_doc['pdf_pesquisavel'] = pdf_pesquisavel 
    value = {}   
    texto_tratado = texto_extraido(extracted_txt)
    value = define_dados_iniciais(idx, row, row_info, texto_tratado, debug)
    if debug:
        print(f'\n3. funcao: processar_dados_iniciais doc.:{original_file_name} | diretorio: {map_directory} | apos funcao: define_dados_iniciais() value \n{value}\n\n')
    if value:
        nf_dados_doc.update(value)
   


    return nf_dados_doc


# 1.B CABECALHO XXX Funcoes de extracao -cabecalho Raster
def processar_cabecalho_R_PDF(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    data_box_valores = {}
    data_box_conferencia = {}
    data_box_valores['secao'] = section
    
    batch_name_row_info = row_info.get('batch')
    #status_documento_row_info = row_info.get('status_documento')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    # Busco a imagem np do documento
    image_np_row_info = row_info.get('image_np')
    
    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info
    data_box_valores['processo'] = context_mapping
    data_box_valores['conf_cod'] = 0


                     
    
    # busco coordenadas para o contexto
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"
    
    #print(f'\n2. Dentro func: section: {section} mapping_method: {mapping_method} | context_mapping: {context_mapping} | model_map: {model_map} | original_file_name: {original_file_name}\n')
   
    # 2. usando a funcao de extracao de coordenadas por contexto    
    coordinates = get_coordinates_filter_by_context(pdf_pesquisavel_map, model_map, context_mapping, tipo_4_coordinates)
    x0, y0, x1, y1 = coordinates[0]
    #print(f'x0: {x0} | y0: {y0} | x1: {x1} | y1: {y1}')
    x0 = int(x0)
    y0 = int(y0)
    x1 = int(x1)
    y1 = int(y1) 
    # 3. Cropo a imagem - novo modelo
    cropped_image_np = image_np_row_info[y0:y1, x0:x1] # ajustar nos demais
    data_box_conferencia[f'box_{context_mapping}'] = cropped_image_np
    data_box_conferencia[f'coordinates_{context_mapping}'] = coordinates
    # 4. Converto para PIL
    cropped_image_pil = Image.fromarray(cropped_image_np)
    # 6. Executo OCR
    texto_extraido = pytesseract.image_to_string(cropped_image_pil, lang='por')
    # 7. Trato o texto extraido = text_splited
    text_splited = texto_extraido_cabecalho(texto_extraido)
    if debug:
        print()
        plt.imshow(cropped_image_np)
        plt.axis('off')  # Desativa os eixos para uma visualização mais limpa
        plt.show()
        print(f'\ncoordinates {coordinates} - \ntexto_extraido:\n{text_splited}\n')
        
    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]
    
    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        try:
            section = row_frame['section_json']
            label = row_frame['label']
            reference = row_frame['reference']
            string_pesquisa = row_frame['marcador_inicio']  
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_box_valores[label] = texto
            if debug:
               print(f'\nidx: {index_frame:> 3} | label: {label} |  string_pesquisa:{string_pesquisa} | dentro do try do raster PDF cabecalho - texto: \n{texto}\n\n')
        except Exception as e:
            msg = (f"{e}")
            data_box_conferencia[label] = msg
    

    # Verificações após o loop
    for key, value in data_box_valores.items():
        if key == 'numero_nota_fiscal' and value is None:
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'Número da Nota não encontrado'
            #logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir
        
        elif key == 'codigo_verificacao' and value != None:
            codigo_verificacao_nf = value
            tam_codigo_verificacao = len(codigo_verificacao_nf)
            data_box_valores['conf_cod'] = tam_codigo_verificacao
            
        
        elif key != 'numero_nota_fiscal' and value is None:
            logging.error(f" {batch_name_row_info} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

            
      # if value is None:
        #     logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info

    
    return data_box_valores

# 1.A CABECALHO - PDF PESQUISAVEL  
def extrai_cabecalho_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_pm, model_map, f_0, f_1, original_file_name, file_path, debug):
    
    nf_data_cabecalho = {}
    lista_erros = []
    label = "1_frame_dados_nf"
    
    batch_name_row_info = row_info.get('batch')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    nf_data_cabecalho['secao'] = section
    nf_data_cabecalho['action_item'] = action_item_row_info
    nf_data_cabecalho['informations'] = information_row_info
    nf_data_cabecalho['processo'] = 'mapeamento regex - PDF pesquisavel'
    
    if debug:
        print(f'\n\n2. dentro da funçao extrai_cabecalho_PDF: batch_name: {batch_name_row_info}\n\n')
    
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]    
    tipo = "frame"

    coordinates = get_coordinates_filter(pdf_pesquisavel_map=pdf_pesquisavel_map, model=model_map, tipo=tipo, label=label, section=section)
    x0, y0, x1, y1 = coordinates[0]
    y0 = y0 * f_0
    y1 = y1 * f_1
    
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    if debug:
        print(f'\n3. x0: {x0}, y0: {y0}, x1: {x1}, y1: {y1} f_0: {f_0} f_1: {f_1} | text: \n{text} \n\n')

    try:
        numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
        if numero_nota_match:
            numero_nf = numero_nota_match.group(1)
            nf_data_cabecalho['numero_nota_fiscal'] = numero_nf
            #nf_data_cabecalho['informations'] = 'documento com numero de nota fiscal'
            if debug:
                print(f'\nnr_nro_nf: {nr_nro_nf} - doc: {original_file_name}\n')
        else:
            msg = (f"Número da Nota não encontrado")
            nf_data_cabecalho['numero_nota_fiscal'] = None
            information_row_info = 'Número da Nota não encontrado'
            nf_data_cabecalho['informations'] = information_row_info
            action_item_row_info = 'BREAK_PROCESS'
            nf_data_cabecalho['action_item'] = action_item_row_info
    except Exception as e:
        msg = (f"doc: {original_file_name} | numero NF nao encontrado {e}")
        nf_data_cabecalho['numero_nota_fiscal'] = None
        information_row_info = 'Número da Nota não encontrado'
        nf_data_cabecalho['informations'] = information_row_info
        action_item_row_info = 'BREAK_PROCESS'
        nf_data_cabecalho['action_item'] = action_item_row_info

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_cabecalho['competencia'] = competencia_match.group(1)

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'Data e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_cabecalho['dt_hr_emissao'] = data_emissao_match.group(1)
        
    # Extrair codigo Verificacao
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        codigo_verificacao_nf = codigo_verificacao_match.group(1)
        nf_data_cabecalho['codigo_verificacao'] =  codigo_verificacao_nf
        tam_codigo_verificacao = len(codigo_verificacao_nf)
        nf_data_cabecalho['conf_cod'] = tam_codigo_verificacao
        
    
    
    pdf_document.close()
    
    return nf_data_cabecalho

## <mark> <b>1.X</b> Funcoes NLP </mark>

In [7]:
def show_ent_new(text, patterns):
    #nlp = spacy.blank("pt")
    #ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    doc = nlp(text)
    
    tokens = []
    ents = []
    
    for ent in doc.ents:
        span = doc.char_span(ent.start_char, ent.end_char, label=ent.label_)
        ents.append(span)
        
    for token in doc:
        start = token.idx
        end = start + len(token)
        tokens.append((token.text, start, end))
        
    return doc, tokens, ents



# chunk.text, chunk.start, chunk.end, chunk.root.head.lemma_, chunk.root.dep_, chunk.doc
def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [8]:
def run_ocrmypdf(input_file, output_file):
    command = [
        'ocrmypdf',
        '--language', 'por',
        '--deskew',
        input_file,
        output_file
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"OCRmyPDF completed successfully. Output saved to {output_file}.")
    else:
        print(f"OCRmyPDF failed with error: {result.stderr.decode('utf-8')}")
        
        
        


# Função para definir o atributo "is_cnpj"
@Language.component("set_cnpj_attribute")
def set_cnpj_attribute(doc):
    for i, token in enumerate(doc):
        if i < len(doc) - 1:
            next_token = doc[i + 1]
            if token.shape_ == "dd.ddd.ddd/" and next_token.shape_ == "dddd-dd":
                token._.is_cnpj = True
                next_token._.is_cnpj = True
            else:
                token._.is_cnpj = False
    return doc        


# Registro do atributo 'is_cnpj'
Token.set_extension('is_cnpj', force=True, default=False)


# Função para aplicar o matcher
@Language.component("apply_cnpj_matcher")
def apply_cnpj_matcher(doc):
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        for token in span:
            token._.is_cnpj = True
    return doc




In [9]:
# 0.A Extracao de texto de todo o documento - PDF PESQUISAVEL	
def extrai_texto_PDF_P(idx, row, row_info, section, map_directory, original_file_name, file_path, debug):    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Extrair texto dentro do retângulo
    text_P = page.get_text("text")
    
    pdf_document.close()
    
    texto_PDF_P = text_P.replace('\n', ' ') 
    if debug:
        print(f'\nFUNC extrai_texto_PDF_P: doc.:{original_file_name} | diretorio: {map_directory}  texto_PDF_P: \n\n{texto_PDF_P}\n\n')

    return texto_PDF_P

In [ ]:
nlp.add_pipe("set_cnpj_attribute") # Adicione esta etapa se você quiser definir o atributo manualmente

nlp.add_pipe("apply_cnpj_matcher")  # Adicione esta etapa para aplicar o matcher

### Matcher Patterns

In [69]:
matcher = Matcher(nlp.vocab)

# Matcher Patterns
#======================================== 1. CABECALHO
# 1. Número da Nota:
numero_nota_pattern = [
    {"LOWER": "número"},
    {"LOWER": "da"},
    {"LOWER": "nota"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True}
]
matcher.add("numero_nota_fiscal", [numero_nota_pattern])


# 2. Competência:
competencia_pattern = [
    {"LOWER": "competência"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "?"},
    {"ORTH": {"REGEX": "^[A-Z][a-z]+/[0-9]{4}$"}}   
]    
matcher.add("competencia", [competencia_pattern])

# 3. Data e Hora de Emissão:
data_hora_emissao_pattern = [
    {"LOWER": "data"},
    {"LOWER": "e"},
    {"LOWER": "hora"},
    {"LOWER": "da"},
    {"LOWER": "emissão"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd/dd/dddd"},
    {"IS_SPACE": True, "OP": "*"},
    {"SHAPE": "dd:dd:dd"}
]
matcher.add("dt_hr_emissao", [data_hora_emissao_pattern])

# 4. Código de Verificação:
codigo_verificacao_pattern = [
    {"LOWER": "código"},
    {"LOWER": "verificação"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ASCII": True, "LENGTH": 9}
]
matcher.add("codigo_verificacao", [codigo_verificacao_pattern])




#========================================  5. VALOR TOTAL
valor_total_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "total"},
    {"LOWER": "da", "OP": "?"},
    {"LOWER": "nota", "OP": "?"},
    {"TEXT": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_TOTAL", [valor_total_pattern])





#======================================== 7. VALORES E IMPOSTOS
# 1. VALOR_SERVICOS
valor_servicos_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "serviços"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]

matcher.add("VALOR_SERVICOS", [valor_servicos_pattern])


# 2. VALOR DEDUÇÃO:
valor_deducao_pattern = [
    {"LOWER": "dedução"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]

matcher.add("VALOR_DEDUCAO", [valor_deducao_pattern])



# 3. DESC. INCOND: RASTER_PDF
valor_incondR_pattern = [
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"IS_SPACE": True},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"ORTH": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}    
]
matcher.add("VALOR_INCONDP", [valor_incondR_pattern])


# 3.A DESC. INCOND: - PDF_Pesquisavel   #DESC. INCOND:
valor_incond_patternP = [
    {"LOWER": "desc"},
    {"IS_PUNCT": True, "OP": "?"},
    {"LOWER": "incond"},
    {"ORTH": ":"},
    {"IS_SPACE": True},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"ORTH": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}    
]
matcher.add("VALOR_INCONDR", [valor_incond_patternP])



# 4. BASE DE CÁLCULO:  RASTER_PDF
valor_calculoR_pattern = [
    {"LOWER": "cálculo"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_CALCULOR", [valor_calculoR_pattern])


# 4.A BASE DE CÁLCULO:  PDF_P
valor_calculoP_pattern = [
    {"LOWER": "base"},
    {"LOWER": "de"},
    {"LOWER": "cálculo"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ",", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_CALCULOP", [valor_calculoP_pattern])



# 5. Alíquota d,dd
valor_aliquota_pattern = [
    {"LOWER": "alíquota"},
    {"ORTH": ":"},
    {"SHAPE": "d,dd", "OP": "?"},
    {"ORTH": "%"}

]
matcher.add("VALOR_ALIQUOTA", [valor_aliquota_pattern])

# 5.1 Alíquota d
valor_aliquota2_pattern = [
    {"LOWER": "alíquota"},
    {"ORTH": ":"},
    {"SHAPE": "d", "OP": "?"},
    {"ORTH": "%"}

]
matcher.add("VALOR_ALIQUOTA2", [valor_aliquota2_pattern])



# 6. VALOR ISS:
valor_iss_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_ISS", [valor_iss_pattern])



# 7. VALOR ISS RETIDO:
valor_issretido_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"LOWER": "retido"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_ISSRETIDO", [valor_issretido_pattern])


# 8. DESC. COND:
valor_desccond_pattern = [
    {"LOWER": "desc"},
    {"ORTH": "."},
    {"LOWER": "cond"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_DESCCOND", [valor_desccond_pattern])


# 9. VALOR PIS:
valor_pis_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "pis"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_PIS", [valor_pis_pattern])


# 10. VALOR COFINS:
valor_cofins_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "cofins"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_COFINS", [valor_cofins_pattern])




# 11. VALOR IR:
valor_ir_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "ir"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_IR", [valor_ir_pattern])


# 12. VALOR INSS:
valor_inss_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "inss"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_INSS", [valor_inss_pattern])


# 13. VALOR CSLL:
valor_csll_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "csll"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_CSLL", [valor_csll_pattern])



# 14. OUTRAS RETENÇÕES:
valor_outrasreten_pattern = [
    {"LOWER": "outras"},
    {"LOWER": "retenções"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_OUTRAS", [valor_outrasreten_pattern])



# 15. VALOR LÍQUIDO:
valor_liquido_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "líquido"},
    {"ORTH": ":"},
    {"SHAPE": "X$"},
    {"MORPH": "NumType=Card", "OP": "+"},
    {"LOWER": ".", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"}
]
matcher.add("VALOR_LIQUIDO", [valor_liquido_pattern])



#======================================== 9. OUTRAS INFORMAÇOES / CRITICAS
# 1. EXIGIBILIDADE ISS
exigibilidade_iss_pattern = [
    {"LOWER": "exigibilidade"},
    {"LOWER": "iss"},
    {"LOWER": {"IN": ["exigivel", "não exigivel"]}}
]
matcher.add("EXIGIBILIDADE_ISS", [exigibilidade_iss_pattern])


# 2. REGIME TRIBUTAÇÃO
padrao_regime_tributacao = [
    {"LOWER": "regime"},
    {"LOWER": "tributação"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ALPHA": True, "OP": "+"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_ALPHA": True, "OP": "*"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": "simples", "OP": "?"},
    {"IS_ALPHA": True, "OP": "*"}
]
matcher.add("REGIME_TRIBUTACAO", [padrao_regime_tributacao])

# 3. SIMPLES NACIONAL = NAO
simples_nacional_nao_pattern = [
    {"LOWER": "simples"},
    {"LOWER": "nacional"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": "não"}
]
matcher.add("SIMPLES_NACIONAL_NAO", [simples_nacional_nao_pattern])

# 3.1 SIMPLES NACIONAL = SIM
simples_nacional_pattern = [
    {"LOWER": "simples"},
    {"LOWER": "nacional"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": "sim", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": "(", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True, "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": ",", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True, "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": "%", "OP": "?"},
    {"IS_SPACE": True, "OP": "*"},
    {"ORTH": ")", "OP": "?"}
]
matcher.add("SIMPLES_NACIONAL_SIM", [simples_nacional_pattern])


# 4. ISSQN RETIDO
issqn_retido_pattern = [
    {"LOWER": "issqn"},
    {"LOWER": "retido"},
    {"IS_SPACE": True, "OP": "*"},
    {"LOWER": {"IN": ["sim", "não"]}}
]
matcher.add("ISSQN_RETIDO", [issqn_retido_pattern])


# 5. LOCAL. PRESTAÇÃO SERVIÇO
local_prestacao_servico_pattern = [
    {"LOWER": "local"},
    {"ORTH": "."},
    {"LOWER": "prestação"},
    {"LOWER": "serviço"},
    {"IS_SPACE": True, "OP": "+"},  # para lidar com múltiplos espaços
    {"IS_ALPHA": True, "OP": "+"},  # para a cidade
    {"ORTH": "-", "OP": "?"},
    {"IS_UPPER": True, "LENGTH": 2, "OP": "?"}  # para a sigla do estado
]
matcher.add("LOCAL_PRESTACAO_SERVICO", [local_prestacao_servico_pattern])

# 6. LOCAL INCIDÊNCIA
local_incidencia_pattern = [
    {"LOWER": "local"},
    {"IS_PUNCT": True, "OP": "?"},
    {"LOWER": "incidência"},
    {"IS_ALPHA": True, "OP": "+"},  # Nome da cidade
    {"ORTH": "-", "OP": "?"},  # Hífen opcional
    {"SHAPE": "XX", "OP": "?"}  # Sigla do estado
]
matcher.add("LOCAL_INCIDENCIA", [local_incidencia_pattern])


# observacao_pattern = [
#     {"LOWER": "observação"},
#     {"ORTH": ":"},
#     {"IS_SPACE": True, "OP": "*"},
#     {"LOWER": "-", "OP": "?"},
#     {"IS_PRINT": True, "OP": "+"}
# ]

# matcher.add("OBSERVACAO", [observacao_pattern])# 6. Alíquota
valor_aliquota_pattern = [
    {"LOWER": "valor"},
    {"LOWER": "iss"},
    {"ORTH": ":"},
    {"IS_SPACE": True, "OP": "*"},
    {"IS_DIGIT": True, "OP": "+"},
    {"ORTH": "", "OP": "?"},
    {"IS_DIGIT": True, "OP": "*"},
    {"ORTH": "%"}
]



colors = {
            "secretaria": "linear-gradient(90deg, #2ADB5E, #1FA346)", # Verde Degrade
            "tipo_documento": "linear-gradient(90deg, #09D6FF, #08A0D1)", #Azul medio degrade
            "nome_prefeitura": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", # Roxo claro para lilaz - degrade bem bacana
            "nome_section": "linear-gradient(90deg, #FFA9FB, #BF7FBC)", #  lilaz - Degrade
            "nome_section": "#FFEA7F", # Laranja claro
            "SAFRA": "#CCA10C", # Terracota
            "SAFRA": "#AB9BFC", # Roxo claro 
            "INTENT": "#7AECEC", # Azul bem claro
            "NOME": "#EE8AF8" # Rosa medio
        }          

patternsPrefeitura = [
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mesquita"}], "id": "PM_MESQUITA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mage"}], "id": "PM_MAGE"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "de"}, {"LOWER": "aldeia"}], "id": "PM_SPA"}

                        ]


patternsSection = [
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "discriminação"}, {"LOWER": "dos"}, {"LOWER": "serviços"}], "id": "4. DESCRIMINACAO DOS SERVIÇOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "dados"}, {"LOWER": "complementares"}], "id": "8. DADOS COMPLEMENTARES"},

                        ]


patternsSecretarias = [{"label": "secretaria", "pattern": [{"LOWER": "secretaria"}, {"LOWER": "municipal"}, {"LOWER": "da"}, {"LOWER": "fazenda"},], "id": "SECRETARIA"}] 


patternsTipoDocumento = [
                        {"label": "tipo_documento", "pattern": [{"LOWER": "nota"}, {"LOWER": "fiscal"}, {"LOWER": "de"}, {"LOWER": "serviços"}, {"LOWER": "eletrônica"}, {"LOWER": "-"}, {"LOWER": "nfs-e"}], "id": "NFS-e"}
                        ]




patternsTemp = [
                    {"label":"TOTAL", "pattern": [{"LOWER": "quantidade", "OP":"?"}, {"LOWER": "total", "OP":"*"}], "id": "qtde-total"},
                    {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entregue","OP":"*"}],"id": "qtde-entregue"},
                    {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entreguei","OP":"*"}],"id": "qtde-entregue"},
                    {"label":"ENTREGUE", "pattern": [{"LOWER": "foram", "OP":"?"},{"LOWER": "entregues","OP":"*"}],"id": "qtde-entregue"},
                    {"label":"SALDO", "pattern": [{"LOWER": "saldo","OP":"*"}], "id": "qtde-saldo"}]


patternsOthers = [{"label": "PERSON", "pattern": "Daniel", "id": "pessoa-daniel"}] 
 
patternsCult = [
    {
        "label":"CULTURA",
        "pattern": [
            {"LOWER": "soja", "OP":"?"},
            {"LOWER": "milho", "OP":"?"},
            {"LOWER": "sorgo", "OP":"?"},
            {"LOWER": "trigo", "OP":"?"},
            {"LOWER": "milheto", "OP":"?"},  
            
        ],    
    "id": "cultura"}]

patternsQuant = [{"label":"TOTAL", "pattern": [{"LOWER": "quantidade", "OP":"?"}, {"LOWER": "total", "OP":"*"}], "id": "qtde-total"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entregue","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "quantidade", "OP":"?"},{"LOWER": "entreguei","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"ENTREGUE", "pattern": [{"LOWER": "foram", "OP":"?"},{"LOWER": "entregues","OP":"*"}],"id": "qtde-entregue"},
                 {"label":"SALDO", "pattern": [{"LOWER": "saldo","OP":"*"}], "id": "qtde-saldo"}]


patternsSafra = [{"label":"SAFRA", "pattern": [{"LOWER": "safra", "OP":"?"},{"LOWER": "safras", "OP":"?"}], "id": "safra"}]


patternsNroSafra = [{"label":"NR_SAF", "pattern": [{"SHAPE": "dd/dd", "OP":"*"}], "id": "nro_safra"},
                    {"label":"NR_SAF", "pattern": [{"lower": "próxima", "OP":"*"}], "id": "nro_safra"},
                    {"label":"NR_SAF", "pattern": [{"lower": "passada", "OP":"*"}], "id": "nro_safra"}]   

patternsCliente = [{"label": "CLIENTE", "pattern": [{"LOWER": "berdinazzi"}], "id": "cli-berdinazzi"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "lopito"}], "id": "cli-lopito"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "bungue"}], "id": "cli-bungue"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["bunge", "bongue", "bumgue"]}}}], "id": "cli-bungue"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["berdinazi"]}}}], "id": "cli-berdinazzi"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["matarazzo"]}}}], "id": "cli-matarazzo"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["mezenga"]}}}], "id": "cli-mezenga"},
                   {"label": "CLIENTE", "pattern": [{"LOWER": "rei"}, {"LOWER": "do"}, {"LOWER": "gado"}], "id": "cli-reidogado"},
                   {"label": "CLIENTE", "pattern": [{"TEXT": {"FUZZY": {"IN": ["rei-do-gado"]}}}], "id": "cli-reidogado"},   
                   ]


patternsContrato = [{"label": "CONTRATO", "pattern": [{"LOWER": "contrato", "OP":"?"}], "id": "contrato"},
                    {"label": "CONTRATO", "pattern": [{"LOWER": "contratos", "OP":"?"}], "id": "contrato"}]

patternsNroContrato = [{"label": "NR_CONT", "pattern": [{"SHAPE": "dddX", "OP":"*"}], "id": "nro_contrato"},
                       {"label": "NR_CONT", "pattern": [{"POS": "NUM", "SHAPE": "ddd", "OP": "*"},
                                                            {"POS": "PROPN", "SHAPE": "X", "OP": "*"}], "id": "nro_contrato"}]

patternsIntent = [{"label": "INTENT", "pattern": [{"IS_TITLE": True, "OP":"*"}], "id": "user-intent"},
                  {"label": "INTENT", "pattern": [{"POS": "ADJ", "OP":"*"}, {"POS": "VERB", "OP":"*"}], "id": "user-intent"},
                  {"label": "INTENT", "pattern": [{"TEXT": {"FUZZY": {"IN": ["preciso", "gostaria", "informar"]}}}], "id": "user-intent"}]


patterns = patternsPrefeitura + patternsSection + patternsSecretarias + patternsTipoDocumento 


In [ ]:
valor = 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e'
valor.lower()

## 2.0 - Processo de Extracao

### <b>2.x</b> Templates e Dics 

In [11]:
def define_dados_iniciais(idx, row, row_info, texto_tratado, debug):
    
    dados_iniciais_nf = {}
    #status_documento_row_info = row_info.get('status_documento')
    action_item_row_info = row_info.get('action_item')
    information_row_info = row_info.get('informations')
    
    dados_iniciais_nf['action_item'] = action_item_row_info
    dados_iniciais_nf['informations'] = information_row_info
    
    print(f'\nDentro da func define_dados_iniciais:  -action_item_row_info: {action_item_row_info}')
   


    prefeitura_encontrada = None
    de_para_encontrado = None

    # 7. ZZZ Dicionário para mapear Prefeitura com sua sigla
    de_para_prefeitura = {
        "PREFEITURA DA CIDADE MAGE": "PM_MAGE",
        "PREFEITURA DA CIDADE DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA\nALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE MESQUITA": "PM_MESQUITA",
        "PREFEITURA MUNICIPAL DE DE MESQUITA": "PM_MESQUITA",
        # ... adicione 
    }
    

    templates = {
        ("PM_MAGE", None): "MAGE",
        ("PM_SPA", None): "SPA",
        ("PM_MESQUITA", None): "MESQUITA",
        ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
        # ... adicione outras combinações aqui
    }

    cnpj_encontrado = None
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for pref in de_para_prefeitura.keys():
            if pref in linha:
                #print(linha)
                prefeitura_encontrada = pref
                dados_iniciais_nf['prefeitura'] = prefeitura_encontrada
                if debug:
                    print(f'\n4.funcao: define_dados_iniciais(texto_tratado) - dentro do loop for de pesquisa prefeitura - prefeitura_encontrada: \n{prefeitura_encontrada}\n\n')
    # Saímos do loop, agora vamos verificar qual template usar
    if prefeitura_encontrada:
        de_para_pm = de_para_prefeitura.get(prefeitura_encontrada)
        dados_iniciais_nf['de_para_pm'] = de_para_pm
        if debug:
            print(f'\n5.funcao: define_dados_iniciais(texto_tratado) - if prefeitura_encontrada - de_para_pm \n{de_para_pm}\n\n')
        if not de_para_pm:
            de_para_pm = de_para_prefeitura.get(prefeitura_encontrada, "NAO_PROCESSAR")
            dados_iniciais_nf['de_para_pm'] = de_para_pm
            #print(de_para_pm)
    else:
        de_para_pm = "NAO_PROCESSAR"
        action_item_row_info = 'BREAK_PROCESS'
        information_row_info = 'Nao identificado dados iniciais para o documento'
        
     
        
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for de_para, cnpj in templates.keys():
            if cnpj and cnpj in linha:
                cnpj_encontrado = cnpj
                dados_iniciais_nf['cnpj_encontrado'] = cnpj_encontrado
                
                
    # Saímos do loop, agora vamos verificar qual template usar
    if de_para_pm:
        template_usar = templates.get((de_para_pm, cnpj_encontrado))
        logging.info(f'usara template {template_usar} para: {cnpj_encontrado}')
        # print(template_usar)
        dados_iniciais_nf['model'] = template_usar
        if not template_usar:
            template_usar = templates.get((de_para_pm, None), "TEMPLATE_NAO_ENCONTRADO")
            dados_iniciais_nf['model'] = 'NAO_ENC.' 
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'model nao encontrado'
    else:
        template_usar = "TEMPLATE_NAO_ENCONTRADO"
        dados_iniciais_nf['model'] = 'NAO_ENC.'
        action_item_row_info = 'BREAK_PROCESS'
        information_row_info = 'model nao encontrado'
        
        
        
        
    
    #Confirmando se template existe em frames    
    try:        
        f_type = 'frame'
        #template_usar = 'SAO_PEDRO_SUPERMIX'
        result = filtrar_df(frames_nf_v4_df, type=f_type, de_para_pm=de_para_pm, model=template_usar)
        model = result['model'].values[0]
        if model:
            template_oficial = model
            if model == template_usar:
                dados_iniciais_nf['model'] = template_oficial
            else:    
                template_usar = "necessario cadastrar"
                dados_iniciais_nf['model'] = "CADASTRAR"
                
            dados_iniciais_nf['model'] = template_usar
        else:
            template_usar = "necessario cadastrar"
            dados_iniciais_nf['model'] = "CADASTRAR"

                
    except Exception as e:
       error_msg = (f"Erro busca do template: {e}") 
    
    dados_iniciais_nf['action_item'] = action_item_row_info 
    dados_iniciais_nf['informations'] = information_row_info         
        
    return dados_iniciais_nf  



nf_model_path = "config/modelos/frames_nf_v11.xlsx"

#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)


# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

ver = tmod.get_template_version(frames_nf_v4_df, 'MAGE')

frames_nf_v4_df.head(5)

print(f'frames_nf_v4_df: {ver}')
print()

frames_nf_v4_df: 2.5



In [ ]:
# 1. XXX Buscar proximo Batch caso nao esteja rodando email
batch_name = utl.busca_proximo_batch(conf_export_plan_path)

# 2. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")


#3. XXX Ler a planilha e cria df_documento_recebido
df_root_pipe = pd.read_excel(file_path_root_pipe)


#4. XXX  Ajustar o indice
df_root_pipe.set_index('document_unique_id', inplace=True)


df_root_pipe.head(5)

### <mark> <b>2.x</b> ExecuÇao do Pipeline de Extracao </mark>

In [ ]:
def processar_cabecalho_PDF_P(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    
    data_box_valores = {}
    data_box_conferencia = {}
    data_box_valores['secao'] = section
    
    batch_name_row_info = row_info.get('batch')
    #status_documento_row_info = row_info.get('status_documento')
    information_row_info = row_info.get('informations')
    action_item_row_info = row_info.get('action_item')
    
    # Busco a imagem np do documento
    image_np_row_info = row_info.get('image_np')
    
    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info
    data_box_valores['processo'] = context_mapping
    data_box_valores['conf_cod'] = 0


    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]
    
    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        try:
            section = row_frame['section_json']
            label = row_frame['label']
            reference = row_frame['reference']
            string_pesquisa = row_frame['marcador_inicio']  
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_box_valores[label] = texto
            if debug:
               print(f'\nidx: {index_frame:> 3} | label: {label} |  string_pesquisa:{string_pesquisa} | dentro do try do raster PDF cabecalho - texto: \n{texto}\n\n')
        except Exception as e:
            msg = (f"{e}")
            data_box_conferencia[label] = msg
    

    # Verificações após o loop
    for key, value in data_box_valores.items():
        if key == 'numero_nota_fiscal' and value is None:
            action_item_row_info = 'BREAK_PROCESS'
            information_row_info = 'Número da Nota não encontrado'
            #logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir
        
        elif key == 'codigo_verificacao' and value != None:
            codigo_verificacao_nf = value
            tam_codigo_verificacao = len(codigo_verificacao_nf)
            data_box_valores['conf_cod'] = tam_codigo_verificacao
            
        
        elif key != 'numero_nota_fiscal' and value is None:
            logging.error(f" {batch_name_row_info} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

            
      # if value is None:
        #     logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")  # Ou registre o erro de outra forma que preferir

    data_box_valores['action_item'] = action_item_row_info
    data_box_valores['informations'] = information_row_info

    
    return data_box_valores

In [125]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [ ]:

# 9. iterar sobre o filtro
for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
    # ... (seu código anterior)

    raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)
    
    # Encontrar o marcador mais semelhante no valor bruto
    most_similar_reference = max([reference], key=lambda x: similar(x, raw_value))
    
    # Substituir o marcador mais semelhante
    final_value = raw_value.replace(most_similar_reference, "").strip()
    
    print(f'\nidx: {index_frame:> 3} | label: {label} |  reference: {most_similar_reference} - raw_value: {raw_value} | final_value: {final_value} ')


In [90]:
file_path = 'pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_23/MESQUITA_PDF_31282023_2258/159871/2023 -4.pdf'
original_file_name = os.path.basename(file_path)

In [98]:
section = "1. CABECALHO"
mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
context_mapping = "data_cabecalho"
def_replace = True 
model_map = 'MESQUITA'



In [132]:
def processar_cabecalho_PDF_P(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, original_file_name, file_path, debug):
    data_box_valores = {}
    if mapping_method == "frame_&_sframe_field":
        tipo_4_coordinates = "frame"
        tipo_4_filter = "sframe_field"

    # 8. Efetuo o filtro para a iteracao
    filtered_frame_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['context_mapping'] == context_mapping) & (frames_nf_v4_df['type'] == tipo_4_filter)]

    # 9. iter sobre o filtro
    for index_frame, row_frame in filtered_frame_nf_v4_df.iterrows():
        section = row_frame['section_json']
        label = row_frame['label']
        reference = row_frame['reference']
        string_pesquisa = row_frame['marcador_inicio']  
        
        raw_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == label), None)
        
        most_similar_reference = max([reference], key=lambda x: similar(x, raw_value))
        ##print(f'\nmost_similar_reference: {most_similar_reference}\n')
        final_value = raw_value.split(":", 1)[-1].strip()
        data_box_valores[label] = final_value

    return data_box_valores        
        

In [131]:
data_box_valores

{'numero_nota_fiscal': '20234',
 'competencia': 'Julho/2023',
 'dt_hr_emissao': '27/07/2023 15:11:00',
 'codigo_verificacao': '4FDA9FBAE'}

In [110]:
numero_nota_fiscal_value = next((doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == 'numero_nota_fiscal'), None)
numero_nota_fiscal_value

'Número da Nota: 20234'

In [139]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status, debug=False, prestador=True, tomador=True, servicos=True, total=True, cnae=True, valores_impostos=True, complementares=True, outras_informacoes=True, observacoes=True):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []
    time_now = cron.timenow_pt_BR()
    func_fase = fase
    func_atividade = atividade
    func_status = status
    lista_dicts = []
    conf_processo = {}
    lista_conferencia = []
   
    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        map_seq = row['seq']
        map_batch_name = row['batch']
        map_fase_processo = row['fase_processo']
        map_nome_atividade = row['nome_atividade']
        map_status_documento = row['status_documento']
        map_original_file_name = row['original_file_name']
        map_directory = row['directory']
        map_one_page = row['one_page']
        map_palavra_chave = row['palavra_chave']
        map_document_tag = row['document_tag']
        map_action_item = row['action_item']
        map_level = row['level']
        file_path = row['file_path']
        row_info['document_unique_id'] = map_document_unique_id
    
        # XXX Nivel 1 - Definindo que documentos serao tratados    
        if map_status_documento == 'PREPROCESS_EXTRACT':
            
            action_item_row_info = 'CONTINUE_PROCESS'
            row_info['action_item'] = action_item_row_info
            information_row_info = 'iniciado processamento'
            row_info['informations'] = information_row_info
            # 0. DADOS GERAIS DOCUMENTO
            section = "0. DADOS INICIAIS"
            try:
                valores = {}
                # 1. XXX Extracao de todos os dados do documento PDF Pesquisavel
                texto_PDF = extrai_texto_PDF_P(idx, row, row_info, section, map_directory, map_original_file_name, file_path, debug)
                if texto_PDF:
                    pdf_pesquisavel_map = True
                else:
                    ppdf_pesquisavel_mapl = False
                
                # 2. XXX IMPORTANTE - Efetuo a busca de entidades e efetuo a tokenizaÇao do documento
                doc, tokens, ents = show_ent_new(texto_PDF, patterns=patterns)
                
                prefeitura_map = [ent.orth_ for ent in doc.ents if ent.label_ == "nome_prefeitura"][0]
                de_para_pm = [ent.id_ for ent in doc.ents if ent.label_ == "nome_prefeitura"][0]
                secretaria_map = [ent.orth_ for ent in doc.ents if ent.label_ == "secretaria"][0]
                tipo_documento_map = [ent.orth_ for ent in doc.ents if ent.label_ == "tipo_documento"][0]
                
                #print(f'\nNivel 1 - map_seq: {map_seq} | pref.: {prefeitura_map} de_para_pm: {de_para_pm} | documento: {map_original_file_name}')
                f_type = 'document'
                result = utl.filtrar_df(frames_nf_v4_df, type=f_type, de_para_pm=de_para_pm)
                #print(result)
                model = result['model'].values[0]
                #print(f'\nNivel 1 - map_seq: {map_seq} | pref.: {prefeitura_map} de_para_pm: {de_para_pm} | model: {model} | documento: {map_original_file_name}')
            except Exception as e:
                msg = (f'Erro ao processar_dados_iniciais: {e}')
            finally:
                row_info['pdf_pesquisavel'] = pdf_pesquisavel_map
                row_info['texto_PDF'] = texto_PDF
                row_info['doc_PDF'] = doc
                row_info['model'] = model
                row_info['tipo_nota_fiscal'] = tipo_documento_map
                row_info['secretaria'] = secretaria_map
                row_info['prefeitura'] = prefeitura_map
                information_row_info = "Este e apenas um comeco - mas bem comeco mesmo"
                action_item_row_info = 'CONTINUE_PROCESS'
   
            # XXX Nivel 2 - Definindo que os documentos legiveis serao tratados
            if action_item_row_info == 'CONTINUE_PROCESS':
                
                # if not pdf_pesquisavel_map:
                #     # NOVO PROCESSO DE TRATAMENTO DE IMAGEM - Convertendo a imagem para numpy array
                #     if debug:
                #         print("irei gerar a imagem_np")
                #     imagem_gray, image_resized_name = convert_resize_gray(map_original_file_name, file_path, image_resized_path)
                #     imagem_gray_rgb = imagem_gray.convert("RGB")
                #     imagem_gray_np = np.array(imagem_gray_rgb)
                #     row_info['image_np'] = imagem_gray_np
                
                # 1. CABECALHO
                # try:
                section = "1. CABECALHO"
                valores = {}
                mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                context_mapping = "data_cabecalho"
                def_replace = True 
                
                if pdf_pesquisavel_map:
                     valores = processar_cabecalho_PDF_P(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                     row_info.update(valores) 
                # else:
                #     valores = processar_cabecalho_R_PDF(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)   
                #     row_info.update(valores)
         
                # #status_documento_row_info = row_info.get('status_documento')
                # action_item_row_info = row_info.get('action_item')
                # information_row_info = row_info.get('informations')   
                information_row_info = "Este e apenas um comeco"
                action_item_row_info = 'CONTINUE_PROCESS'
                
                # XXX Nivel 3 - Definindo que os documentos legiveis serao tratados realmente
                if action_item_row_info == 'BREAK_PROCESS':
                    #msg = (f'Processo inicial: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory} - information_row_info: {information_row_info}')
                    if debug:
                        print(f'\nINFELIZMENTE - seq: {map_seq} doc: {map_original_file_name} dir: {map_directory} - NAO SERA PROCESSADO  | inf: {information_row_info} \n\n')
               
                    #row_info['informations'] = msg
                    # logging.error(msg)
                    lista_dicts.append(row_info)
                    continue 
                
                    
                elif action_item_row_info == 'CONTINUE_PROCESS':
                    if debug:
                        print(f'\nEBA, BORA CONTINUAR - seq: {map_seq} - proxima section: | PDF Pesquisavel: {pdf_pesquisavel_map} doc: {map_original_file_name} dir: {map_directory} | action_item: {action_item_row_info} | inf: {information_row_info} \n\n')
                        print()
                        print(valores)
                    
                    information_row_info = 'Cabecalho processado'
                    row_info['informations'] = information_row_info
                    
                    
                    # guarda_texto_doc = {}
                    # guarda_texto_doc, linhas = cria_guarda_doc_ref_R_PDF(idx, row, de_para_map, model_map, map_original_file_name, file_path, image_resized_path, debug)
            
                    # 2. PRESTADOR DE SERVIÇO
                    if prestador == True:
                        section = "2. PRESTADOR DE SERVIÇO"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        valores = {}
                        erros_prestador = {}
                        data_tomador = {}
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            valores = extrai_prestador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        else:
                            valores = extrai_prestador_R_PDF(idx, row, row_info, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        
                        if not isinstance(valores, dict):
                            msg_erro = (f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                        else:
                            row_info.update(valores)
                            
                        # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                        # if debug:
                        #     print(msg)
                        # logging.info(msg)
                    
                    # 3. TOMADOR DE SERVIÇO
                    if tomador == True:
                        section = "3. TOMADOR DE SERVIÇO"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        
                        valores = {}
                        erros = []
                        data_tomador = {}
                        f_0 = 1
                        f_1 = 1
                        
                        if pdf_pesquisavel_map:
                            valores = extrai_tomador_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        else:   
                            valores = extrai_tomador_R_PDF(idx, row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                            
                        if not isinstance(valores, dict):
                            print(f"\nErro na linha {idx}: 'valores' não é um dicionário. Tipo: {type(valores)}, Valor: {valores}")
                        else:
                            row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)
                    
                    # 4. DESCRIMINACAO DOS SERVIÇOS
                    if servicos == True:
                        if debug:
                            print(f'processando servicos para: {map_original_file_name}')
                        section = "4. DESCRIMINACAO DOS SERVIÇOS"
                        valores = {}
                        nf_data_servico = {} 
                        f_0 = 1
                        f_1 = 1
                        
                        if pdf_pesquisavel_map:
                            nf_data_servico = processar_servicos_pdf_pesquisavel(row, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                        else:
                            label = "discriminacao_servicos"
                            tipo = "field_box"
                            def_replace = True
                            
                            # ItSs  working
                            texto_extraido = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            row_info[label] = texto_extraido
                            
                        msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                        if debug:
                            print(msg)
                        logging.info(msg)     


                        try:
                            texto_extraido = nf_data_servico['discriminacao_servicos'] 
                            row_info['discriminacao_servicos'] = texto_extraido 
                        except Exception as e:
                            msg = (f"doc: {map_original_file_name} | {e}")
                            discrimanacao_servico = "Descricao nao encontrada"
                            row_info['discriminacao_servicos'] = texto_extraido

                    
                    # 5. VALOR TOTAL
                    if total == True:
                        section = "5. VALOR TOTAL"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        #valores = {}
                        if pdf_pesquisavel_map:
                            valor_total_documento = processar_valor_total_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, model_map, map_original_file_name, file_path, debug)
                            if valor_total_documento:
                                if debug:
                                    print(f'\nvalor_total_documento: {valor_total_documento} | doc: {map_original_file_name}\n')
                                row_info['valor_total_nota'] = valor_total_documento
                        else:
                            label = "valor_total_nota"
                            tipo = "field_box"
                            def_replace = True
                            texto_extraido = extracao_documento_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            if texto_extraido: 
                                valor_total_match = re.search(r'R\$ ([\d,.]+)', texto_extraido)
                                if valor_total_match:
                                    valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                                    try:
                                        # valores['secao'] = section
                                        valor_total_documento = float(valor_total_sem_formatacao)
                                    except Exception as e:
                                        # valores['secao'] = section
                                        valor_total_documento = 0.0
                                        msg = (f'Processo inicial: {batch_name} | {map_original_file_name:>25} | diretorio: {map_directory} | {e}')
                                        #logging.error(f" {batch_name} |  doc: {original_file_name:>25} | setion:{section:20} | item: {key:>20} | erro na extracaçao | file_path: {file_path:>40} ")
                        
                                    if valor_total_documento:
                                        if debug:
                                            print(f'\nvalor_total_documento: {valor_total_documento} | doc: {map_original_file_name}\n')
                                        row_info['valor_total_nota'] = valor_total_documento
         
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)
                    
                    # 6. CNAE e Item da Lista de Serviços 
                    if cnae == True:
                        section = "6. CNAE e Item da Lista de Serviços"
                        data_box_valores = {}
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        f_0_cnae = 0.95
                        f_1_cnae = 1.15
                        f_0_it = 0.95     #0.95
                        f_1_it = 1.15    # 1
                        
                        mapping_method = "frame_&_sframe_field" # significa que as coordenadas estao em frames e os valores dos campos nos sframe_fields
                        context_mapping = "data_cnae"
                        def_replace = True
                        
                        if pdf_pesquisavel_map:
                            data_box_valores = extracao_documento_CNAE_ITEM_PDF_P(idx, row, row_info, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, def_replace, map_original_file_name, file_path, debug)
                        else:
                            data_box_valores = extracao_documento_CNAE_ITEM_R_PDF(idx, row, row_info, guarda_texto_doc, section, mapping_method, context_mapping, pdf_pesquisavel_map, model_map, def_replace, map_original_file_name, file_path, debug)
                            
                        if data_box_valores:
                            row_info.update(data_box_valores)    

                    
                    # 7. VALORES E IMPOSTOS
                    if valores_impostos == True:
                        section = "7. VALORES E IMPOSTOS"
                        # if debug:
                        print(f'processando {section} para: {map_original_file_name} - diretorio: {map_directory}')
                        valores = {}
                        nf_data_valores = {}
                        lista_impostos = []
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            valores = extrai_valores_impostos_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                            row_info.update(valores)
                        
                        else:
                            tipo = "field_box"
                            father_value = "5_frame_valores_impostos"
                            valores = extracao_impostos_R_PDF(section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                            #print(valores)
                            row_info.update(valores)
                        
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg) 
                    
                    # 8. DADOS COMPLEMENTARES
                    if complementares == True:
                        section = '8. DADOS COMPLEMENTARES'
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        nf_data_dados_complementares = {}
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            nf_data_valores = extrai_dados_complementares_PDF_P(idx, row, row_info, section, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path, debug)
                        else:
                            label = "dados_complementares"
                            tipo = "field_box"
                            def_replace = False
                            # ItSs  working
                            texto_extraido = extracao_complementar_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            row_info[label] = texto_extraido  

                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)     
                    
                    # 9. OUTRAS INFORMAÇOES / CRITICAS
                    if outras_informacoes == True:
                        section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')
                        tipo = "field_box"
                        father_value = "5_frame_inf_criticas"
                        valores = {} 
                        nf_data_outras_informacoes = {}
                        f_0 = 1
                        f_1 = 1
                        if pdf_pesquisavel_map:
                            valores = extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                            if valores:
                                row_info.update(valores)
                        else:
                            section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                            tipo = "field_box"
                            father_value = "5_frame_inf_criticas"
                            valores = extracao_inforacoes_criticas_R_PDF(section, tipo, father_value, de_para_map, model_map, map_original_file_name, file_path)
                            if valores:
                                row_info.update(valores)
                            
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)
                    # logging.info(msg)          
                            
                    
                    # 10. OBSERVACOES
                    if observacoes == True:  
                        section = "10. OBSERVACOES"
                        if debug:
                            print(f'processando {section} para: {map_original_file_name}')  
                        data_observacao = {}
                        valores = {}
                        f_0 = 0.9
                        f_1 = 1.1
                        if pdf_pesquisavel_map:
                            valores = extrai_outras_informacoes_PDF_P(row, pdf_pesquisavel_map, de_para_map, model_map, f_0, f_1, map_original_file_name, file_path)
                            if valores:
                                row_info.update(valores)
                        else:
                            section = '10. OBSERVACOES'
                            tipo = "field_box"
                            father_value = "6_section_inf_complementares_criticas" 
                            
                            label = "observacao"
                            tipo = "field_box"
                            def_replace = True
                            valores = extracao_observacoees_R_PDF(idx, row, guarda_texto_doc, section, tipo, label, de_para_map, model_map, def_replace, map_original_file_name, debug)
                            if valores:
                                row_info.update(valores)
                    
                    # msg = (f'secao: {section:>15} processada para: {map_original_file_name} - diretorio: {map_directory}')
                    # if debug:
                    #     print(msg)           
                    
                    
                    
                lista_dicts.append(row_info)
                
                
            elif action_item_row_info == 'BREAK_PROCESS':
                
                msg = (f'Documento sem qualidade para pesquisa inicial: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
                row_info['informations'] = msg  
                
            
                lista_dicts.append(row_info)
                continue
                         

        
        
        
        elif map_status_documento == 'NO_PROCESS':
            msg = (f'Documento nao sera tratado neste escopo: {map_batch_name} | {map_original_file_name} | diretorio: {map_directory}')
            row_info['action_item'] = "NO_PROCESS"    
            row_info['informations'] = msg 
            lista_dicts.append(row_info)
            continue
            
            
            
        
        #lista_dicts.append(row_info)
        
        i += 1

    #logging.info(f'processamento finalizado para: {batch_name}') 
    
    print(f'processamento de {i} documentos')
    
    novo_df = pd.DataFrame(lista_dicts)
    
    #return lista_dicts
    return novo_df

In [140]:
# analisar_pdf_pesquisavel
fase = 'analise'
atividade = 'PREPROCESS' 
status = 'PREPROCESS_EXTRACT'
raw_document_list = []
dados_prest = {}

lista_dicts = []
#logging.info(f'Execuçao do pipeline para {batch_name} | df_root_pipe: {file_path_root_pipe} fase: {fase} atividade: {atividade} status: {status}  template: {ver}')

# 1. Processar somente dados iniciais e cabeçalho
df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

processamento de 14 documentos


In [141]:
df

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,...,texto_PDF,doc_PDF,model,tipo_nota_fiscal,secretaria,prefeitura,numero_nota_fiscal,competencia,dt_hr_emissao,codigo_verificacao
0,6,24/09/2023 09:23:12,Batch_23,analise,scan_analise,PREPROCESS_EXTRACT,Analise,2023 -4.pdf,159871,True,...,"27/07/2023, 15:12 Nota Fiscal de Serviços Elet...","(27/07/2023, ,, 15:12, Nota, Fiscal, de, Servi...",MESQUITA,NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e,SECRETARIA MUNICIPAL DA FAZENDA,PREFEITURA MUNICIPAL DE MESQUITA,20234,Julho/2023,27/07/2023 15:11:00,4FDA9FBAE
1,8,24/09/2023 09:23:12,Batch_23,analise,scan_analise,PREPROCESS_EXTRACT,Analise,2023 -3.pdf,159871,True,...,"27/07/2023, 15:05 Nota Fiscal de Serviços Elet...","(27/07/2023, ,, 15:05, Nota, Fiscal, de, Servi...",MESQUITA,NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e,SECRETARIA MUNICIPAL DA FAZENDA,PREFEITURA MUNICIPAL DE MESQUITA,20234,Julho/2023,27/07/2023 15:11:00,4FDA9FBAE


In [15]:
texto_PDF_Pesquisavel = lista_dicts[0]['content_PDF_P']
texto_PDF_Pesquisavel

"27/07/2023, 15:12 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20234 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:11:00 Código Verificação: 4FDA9FBAE PRESTADOR DE SERVIÇOS CPF/CNPJ:  50.921.369/0001-05 Inscrição Municipal:  952538 Telefone:  2297268232.. Inscrição Estadual:   Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ:  06.047.087/0033-16    |     INSC:MUNICIPAL: RG:   Telefone: Inscrição Estadual:   Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço:  OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Janeiro, 36h no Setor d

In [16]:
doc, tokens, ents = show_ent_new(texto_PDF_Pesquisavel, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [142]:
doc = nlp(texto_PDF_Pesquisavel)

# Executar o Matcher no Doc
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    if string_id == 'numero_nota_fiscal':
        print(f'\nstt: {start:>3} | end:{end:>3} | string_id: {string_id:>30}   span.text:{span.text:>45} ')
    elif string_id == 'competencia':    
        print(f'\nstt: {start:>3} | end:{end:>3} | string_id: {string_id:>30}   span.text:{span.text:>45} ')
    elif string_id == 'dt_hr_emissao':
        print(f'\nstt: {start:>3} | end:{end:>3} | string_id: {string_id:>30}   span.text:{span.text:>45} ')
    elif string_id == 'codigo_verificacao':
        print(f'\nstt: {start:>3} | end:{end:>3} | string_id: {string_id:>30}   span.text:{span.text:>45} ')    
            


stt:  28 | end: 33 | string_id:             numero_nota_fiscal   span.text:                        Número da Nota: 20234 

stt:  33 | end: 36 | string_id:                    competencia   span.text:                      Competência: Julho/2023 

stt:  36 | end: 44 | string_id:                  dt_hr_emissao   span.text:  Data e Hora da Emissão: 27/07/2023 15:11:00 

stt:  44 | end: 48 | string_id:             codigo_verificacao   span.text:                Código Verificação: 4FDA9FBAE 


In [40]:
doc

27/07/2023, 15:12 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20234 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:11:00 Código Verificação: 4FDA9FBAE PRESTADOR DE SERVIÇOS CPF/CNPJ:  50.921.369/0001-05 Inscrição Municipal:  952538 Telefone:  2297268232.. Inscrição Estadual:   Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ:  06.047.087/0033-16    |     INSC:MUNICIPAL: RG:   Telefone: Inscrição Estadual:   Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço:  OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Janeiro, 36h no Setor de

In [ ]:
tokens  

In [80]:
valor = [ent.orth_ for ent in doc.ents if ent.label_ == "tipo_documento"][0]
valor = [ent.orth_ for ent in doc.ents if ent.label_ == "secretaria"][0]
valor
valor

'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e'

In [79]:
valor = [ent.orth_ for ent in doc.ents if ent.label_ == "secretaria"][0]
valor

'SECRETARIA MUNICIPAL DA FAZENDA'

In [78]:
valor = [ent.id_ for ent in doc.ents if ent.label_ == "secretaria"][0]
valor

'SECRETARIA'

In [43]:
prefeitura_encontrada = [ent.orth_ for ent in doc.ents if ent.label_ == "nome_prefeitura"][0]
prefeitura_encontrada

'PREFEITURA MUNICIPAL DE MESQUITA'

In [18]:
for ent in doc.ents:
    print(f'{ent.start:>5} | {ent.text:>50} | {ent.label_:>25} | {ent.id_:>35}  |   {ent.end:>4}   ||   {ent.start_char:>6} | {ent.end_char:>6}')

   13 |                   PREFEITURA MUNICIPAL DE MESQUITA |           nome_prefeitura |                          PM_MESQUIA  |     17   ||       95 |    127
   17 |                    SECRETARIA MUNICIPAL DA FAZENDA |                secretaria |                          SECRETARIA  |     21   ||      128 |    159
   21 |         NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e |            tipo_documento |                               NFS-e  |     28   ||      160 |    202
   48 |                              PRESTADOR DE SERVIÇOS |              nome_section |             2. PRESTADOR DE SERVIÇO  |     51   ||      323 |    344
  109 |                                TOMADOR DE SERVIÇOS |              nome_section |               3. TOMADOR DE SERVIÇO  |    112   ||      660 |    679
  171 |                         DISCRIMINAÇÃO DOS SERVIÇOS |              nome_section |       4. DESCRIMINACAO DOS SERVIÇOS  |    174   ||      931 |    957
  389 |                               DADOS COMPLEME

In [ ]:
fpr token

In [ ]:
#texto_amostra = "NFS-e Número da Nota: 20234 Competência: Julho/2023"
texto_amostra = "NFS-e Número da Nota: 20233 Competência:"

doc = nlp(texto_amostra)
matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id}: {span.text}")

In [ ]:
# Analisys
syntatic = pd.DataFrame(data=[], \
  columns=["id", "T_texto","T_shape", "T_is_alpha", "T_is_digit", "T_is_title", "T_is_punct", "T_is_sent_start", "T_is_right_punct", "T_is_stop", "T_is_quote", "T_is_currency", "T_morph"])
i = 0
for token in doc:
    syntatic.loc[i,"id"] = token.i
    syntatic.loc[i,"T_texto"] = token.text
    syntatic.loc[i,"T_shape"] = token.shape_
    syntatic.loc[i,"T_is_alpha"] = token.is_alpha
    syntatic.loc[i,"T_is_digit"] = token.is_digit
    syntatic.loc[i,"T_is_title"] = token.is_title
    syntatic.loc[i,"T_is_punct"] = token.is_punct
    syntatic.loc[i,"T_is_sent_start"] = token.is_sent_start
    syntatic.loc[i,"T_is_right_punct"] = token.is_right_punct
    syntatic.loc[i,"T_is_stop"] = token.is_stop
    syntatic.loc[i,"T_is_quote"] = token.is_quote
    syntatic.loc[i,"T_is_currency"] = token.is_currency
    syntatic.loc[i,"T_morph"] = token.morph
    i = i+1

syntatic.head(80)

In [45]:
# Lemmatization for tokens 
lemmatization = pd.DataFrame(data=[], \
  columns=["id", "Texto","Lemma", "Tag", "Tag_explainned", "token_POS", "POS_explainned", "dep", "T. Head", "dep explained"])
i = 0
for token in doc:
    lemmatization.loc[i,"id"] = token.i
    lemmatization.loc[i,"Texto"] = token.text
    lemmatization.loc[i,"Lemma"] = token.lemma_
    lemmatization.loc[i,"Tag"] = token.tag_
    lemmatization.loc[i,"Tag_explainned"] = spacy.explain(token.tag_)
    lemmatization.loc[i,"token_POS"] = token.pos_
    lemmatization.loc[i,"POS_explainned"] = spacy.explain(token.pos_)
    lemmatization.loc[i,"dep"] = token.dep_
    lemmatization.loc[i,"T. Head"] = token.head.text
    lemmatization.loc[i,"dep explained"] = token.morph
    
    i = i+1
lemmatization

,id,Texto,Lemma,Tag,Tag_explainned,token_POS,POS_explainned,dep,T. Head,dep explained
0,0,27/07/2023,27/07/2023,NOUN,noun,NOUN,noun,ROOT,27/07/2023,"(Gender=Masc, Number=Sing)"
1,1,",",",",PUNCT,punctuation,PUNCT,punctuation,punct,27/07/2023,()
2,2,15:12,15:12,NUM,numeral,NUM,numeral,punct,27/07/2023,(NumType=Card)
3,3,Nota,Nota,PROPN,proper noun,PROPN,proper noun,ROOT,Nota,(Number=Sing)
4,4,Fiscal,Fiscal,PROPN,proper noun,PROPN,proper noun,flat:name,Nota,(Number=Sing)
...,...,...,...,...,...,...,...,...,...,...
489,489,Alíq,Alíq,PROPN,proper noun,PROPN,proper noun,appos,Tributos,"(Gender=Masc, Number=Sing)"
490,490,IBPT,IBPT,PROPN,proper noun,PROPN,proper noun,flat:name,Alíq,(Number=Sing)
491,491,"2,23","2,23",NUM,numeral,NUM,numeral,nummod,IBPT,(NumType=Card)
492,492,IBPT,IBPT,PROPN,proper noun,PROPN,proper noun,appos,Alíq,"(Gender=Masc, Number=Sing)"


In [ ]:
run_ocrmypdf(input_file, output_file)

In [ ]:
df

In [ ]:
# Criando o subset para analise
df_conf = df[['seq', 'batch', 'original_file_name', 'directory','status_documento', 'model', 'secao', 'prefeitura', 'de_para_pm', 'model', 'action_item', 'pdf_pesquisavel', 'processo', 'numero_nota_fiscal', 'competencia', 'dt_hr_emissao', 'codigo_verificacao','conf_cod' ]]
df_conf

In [ ]:
# Ajustando DF para analises
df.set_index('document_unique_id', inplace=True)

ordem_status = ['PREPROCESS_EXTRACT', 'NO_PROCESS', 'root_analise']
ordem_action_item = ['CONTINUE_PROCESS', 'BREAK_PROCESS', 'NO_PROCESS']


df['status_documento'] = pd.Categorical(df['status_documento'], categories=ordem_status, ordered=True)
df['action_item'] = pd.Categorical(df['action_item'], categories=ordem_action_item, ordered=True)

df.sort_values(by=['status_documento', 'action_item', 'seq'], ascending=[True, True, True], inplace=True)

In [ ]:
# XXX 1.Processar todas as secoes do documento
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=True, tomador=True, servicos=True, total=True, cnae=True, valores_impostos=True, complementares=True, outras_informacoes=True, observacoes=True)



# 5. Processar valor Total
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=True, cnae=False, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

# 6. Processar CNAE
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=True, valores_impostos=False, complementares=False, outras_informacoes=False, observacoes=False)

# 7. Processar Impostos
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=, complementares=False, outras_informacoes=False, observacoes=False)

# 8. complementar e observaçoes
#df = extracao_pipeline(df_root_pipe, fase, atividade, status, debug=False, prestador=False, tomador=False, servicos=False, total=False, cnae=False, valores_impostos=False, complementares=True, outras_informacoes=True, observacoes=True)